In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
app =  pd.read_csv('../data/application_train.csv')[['SK_ID_CURR','TARGET']]
cre=  pd.read_csv('../data/credit_card_balance.csv')
# POS_CASH_balance =  pd.read_csv('../data/POS_CASH_balance.csv')
# installments_payments =  pd.read_csv('../data/installments_payments.csv')

# installments_payments['MONTHS_BALANCE'] = (installments_payments['DAYS_INSTALMENT']/30).round().astype(int)

In [10]:
len(cre['SK_ID_CURR'].unique())

103558

In [11]:
len(app['SK_ID_CURR'].unique())

307511

In [16]:
pd.merge(app, cre, how= 'inner').corr().abs()['TARGET'].sort_values(ascending=False)

TARGET                        1.000000
AMT_BALANCE                   0.050098
AMT_TOTAL_RECEIVABLE          0.049839
AMT_RECIVABLE                 0.049803
AMT_RECEIVABLE_PRINCIPAL      0.049692
AMT_INST_MIN_REGULARITY       0.039798
CNT_DRAWINGS_ATM_CURRENT      0.038437
CNT_DRAWINGS_CURRENT          0.037793
MONTHS_BALANCE                0.035695
CNT_DRAWINGS_POS_CURRENT      0.029536
AMT_DRAWINGS_ATM_CURRENT      0.024700
CNT_INSTALMENT_MATURE_CUM     0.023684
AMT_DRAWINGS_CURRENT          0.022378
AMT_CREDIT_LIMIT_ACTUAL       0.013823
AMT_PAYMENT_CURRENT           0.012929
AMT_PAYMENT_TOTAL_CURRENT     0.012302
SK_DPD_DEF                    0.010538
AMT_DRAWINGS_POS_CURRENT      0.005084
SK_ID_CURR                    0.004617
AMT_DRAWINGS_OTHER_CURRENT    0.003843
CNT_DRAWINGS_OTHER_CURRENT    0.003044
SK_ID_PREV                    0.002571
SK_DPD                        0.001684
Name: TARGET, dtype: float64

In [15]:
df = pd.merge(app,pd.merge(installments_payments, pd.merge(POS_CASH_balance,credit_card_balance, how = 'outer') , how='outer'),how='left').groupby('SK_ID_CURR').mean().corr()['TARGET']

In [19]:
main_features = df.abs().sort_values(ascending=False)[1:9].keys()

In [20]:
main_features

Index(['DAYS_ENTRY_PAYMENT', 'DAYS_INSTALMENT', 'MONTHS_BALANCE',
       'NUM_INSTALMENT_VERSION', 'CNT_INSTALMENT_FUTURE', 'AMT_PAYMENT',
       'AMT_INSTALMENT', 'CNT_INSTALMENT'],
      dtype='object')

# cells bellow will be used to create a model on another notebook

In [ ]:
ap, cre , pos, ins = app.columns,credit_card_balance.columns, POS_CASH_balance.columns, installments_payments.columns

In [ ]:
print([ap, cre , pos, ins].remove(ap))

In [ ]:
ins & pos
    

In [ ]:
POS_CASH_balance.isna().sum().sort_values(ascending=False)

In [ ]:
credit_card_balance = credit_card_balance.rename(columns = {'NAME_CONTRACT_STATUS':'CRE_NAME_CONTRACT_STATUS','SK_DPD':'CRE_SK_DPD', 'SK_DPD_DEF':'CRE_SK_DPD_DEF'})

In [ ]:
len(list(POS_CASH_balance['SK_ID_PREV']) and list(credit_card_balance['SK_ID_PREV']))

In [ ]:
pd.merge(POS_CASH_balance, credit_card_balance, left_on=['SK_ID_PREV'], right_on=['SK_ID_PREV'], how='inner')

In [ ]:
pd.merge(POS_CASH_balance, credit_card_balance, left_on=['SK_ID_PREV'], right_on=['SK_ID_PREV'], how='left').isna().sum().sort_values(ascending=False)

In [ ]:
POS_CASH_balance.head(3)

In [ ]:
credit_card_balance.head(3)

In [ ]:
credit_card_balance.isna().sum().sort_values(ascending=False)

In [ ]:
pd.merge(POS_CASH_balance, credit_card_balance, how='left').shape

In [ ]:
df = pd.merge(app[['SK_ID_CURR', 'TARGET']], POS_CASH_balance, how='left')
print(df.isna().sum().sort_values(ascending = False)[:3],'oioioi' ,df.shape[0])
df = pd.merge(df, credit_card_balance.drop(['SK_ID_CURR'],axis=1), how='left')
print(df.isna().sum().sort_values(ascending = False)[:3], df.shape[0])
# df = pd.merge(df, installments_payments.drop(['SK_ID_CURR'],axis=1), how='left')
# # print(df.isna().sum().sort_values(ascending = False)[:3], df.shape[0])


In [ ]:
credit_card_balance.columns

In [ ]:
df.shape

In [ ]:
df.isna().sum().sort_values(ascending = False)

In [ ]:
y = app['TARGET']

In [ ]:
df = df.drop(['TARGET'], axis=1)

In [ ]:
df = df.pivot_table(values=list(set(df.columns) - set(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE'])), index='SK_ID_CURR', columns='MONTHS_BALANCE')


In [ ]:
df

In [ ]:
df = df.values.reshape((307511,81,4))

In [ ]:
df = np.nan_to_num(df)

In [ ]:
y = y.to_numpy()

In [ ]:
# np.savetxt('../data/X_train.csv', df, delimiter=',')
# np.savetxt('../data/Y_train.csv', y, delimiter=',')

In [ ]:
len(y)

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, random_state=42)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(4, return_sequences = True),
    tf.keras.layers.LSTM(2, return_sequences = False)
])

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error', 
              metrics='accuracy')

In [ ]:
history = model.fit(x=X_train, y=y_train, epochs= 50, batch_size=500, validation_data=(X_test,y_test) )

In [ ]:
loss_train = history.history['train_loss']
loss_val = history.history['val_loss']
epochs = range(1,35)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
cre_fill = credit_card_balance.fillna('mean')

In [ ]:
cre_fill.astype(int)

In [ ]:

print(len(set(app['SK_ID_CURR'])))
print(len(set(credit_card_balance['SK_ID_CURR'])))
print(len(set(POS_CASH_balance['SK_ID_CURR'])))
print(len(set(installments_payments['SK_ID_CURR'])))

In [ ]:
# values =list(set(installments_payments.columns) - set(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE']))

In [ ]:
POS_CASH_balance.columns

In [ ]:
credit_card_balance = credit_card_balance.pivot_table(values=list(set(credit_card_balance.columns) - set(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE'])), index='SK_ID_CURR', columns='MONTHS_BALANCE')

POS_CASH_balance = POS_CASH_balance.pivot_table(values=list(set(POS_CASH_balance.columns) - set(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE'])), index='SK_ID_CURR', columns='MONTHS_BALANCE')

installments_payments = installments_payments.pivot_table(values=list(set(installments_payments.columns) - set(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE'])), index='SK_ID_CURR', columns='MONTHS_BALANCE')

In [ ]:
POS_CASH_balance

In [ ]:
credit_card_balance.shape

In [ ]:
POS_CASH_balance.shape

In [ ]:
priit

In [ ]:
installments_payments.shape[1]/81

In [ ]:
# credit_card_balance= credit_card_balance.values.reshape((86905,80,19))
# POS_CASH_balance= POS_CASH_balance.values.reshape((285213,80,4))
installments_payments= installments_payments.values.reshape((288410,81,6))


In [ ]:
installments_payments.keys

In [ ]:
df = installments_payments[installments_payments['SK_ID_CURR'].isin(app['SK_ID_CURR'])]

In [ ]:
df.shape

In [ ]:
df = pd.merge(df, POS_CASH_balance, how='left')
# df.head(5)

In [ ]:
df.isna().sum()

In [ ]:
df.shape

In [ ]:
df=pd.merge(df, credit_card_balance, how = 'left')
# df.head(5)

In [ ]:
df = df[df['MONTHS_BALANCE']>-80]

In [ ]:
df.shape

In [ ]:

del credit_card_balance, POS_CASH_balance ,installments_payments
gc.collect()

In [ ]:
# df = df[df['SK_ID_CURR'].isin(app['SK_ID_CURR'])]

In [ ]:
values = list(set(df.columns) - set(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE']))

In [ ]:
df_pivot = df.pivot_table(values=values, index='SK_ID_CURR', columns='MONTHS_BALANCE')

In [ ]:
# import pandas as pd
# import numpy as np
# import gc
# credit_card_balance =  pd.read_csv('../data/credit_card_balance.csv')
# POS_CASH_balance =  pd.read_csv('../data/POS_CASH_balance.csv')
# installments_payments =  pd.read_csv('../data/installments_payments.csv')
# installments_payments['MONTHS_BALANCE'] = (installments_payments['DAYS_INSTALMENT']/30).round().astype(int)
# df = pd.merge(POS_CASH_balance, installments_payments,  left_on=(['SK_ID_PREV','SK_ID_CURR','MONTHS_BALANCE']), right_on=(['SK_ID_PREV','SK_ID_CURR','MONTHS_BALANCE']), how='outer')
# df=pd.merge(df, credit_card_balance, how = 'outer')
# del credit_card_balance, POS_CASH_balance ,installments_payments
# gc.collect()
# values = list(set(df.columns) - set(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE']))
# df_pivot = df.pivot_table(values=values, index='SK_ID_CURR', columns='MONTHS_BALANCE').to_json('../treated_data/timeseries.json')